<a href="https://colab.research.google.com/github/veerashayyagari/llms-in-prod/blob/main/Miami_Hotels_Search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"

locale.getpreferredencoding = getpreferredencoding

In [ ]:
!pip install -U openai gradio sentence_transformers accelerate spacy
!python -m spacy download en_core_web_sm

In [3]:
import pandas as pd
url = 'https://raw.githubusercontent.com/hamzafarooq/maven-mlsystem-design-cohort-1/main/data/miami_hotels.csv'
df = pd.read_csv(url)
df.shape

(2511, 27)

In [4]:
df.head()

,id,type,name,image,awards,rankingPosition,priceLevel,priceRange,category,rating,...,prices,latitude,longitude,webUrl,website,rankingString,rankingDenominator,numberOfReviews,review,title
0,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,[],25.807375,-80.12364,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,Hands down my absolute favorite hotel in South...,My favorite hotel in Miami--Petar is the best!
1,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,[],25.807375,-80.12364,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,There are not enough words to explain the sump...,Exquisite Stay
2,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,[],25.807375,-80.12364,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,I went for the Burlesque show which was absolu...,Burlesqu Show
3,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,[],25.807375,-80.12364,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,Dana at Gitano made the night magical! He make...,Dana at Gitano is a rockstar!
4,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,[],25.807375,-80.12364,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,I would like to give our very very heartfelt t...,Perfect Place


In [5]:
df['priceLevel'].value_counts()

$$$     1151
$$$$    1013
$$       300
$         47
Name: priceLevel, dtype: int64

In [6]:
def parse_price_level(level):
  s = 'relative price : '
  if level == '$':
    s += 'inexpensive'
  elif level == '$$':
    s += 'moderately expensive'
  elif level == '$$$':
    s += 'expensive'
  else:
    s += 'very expensive'
  return s

df['formatted_pricelevel'] = df['priceLevel'].apply(lambda x: parse_price_level(x))
df.head()

,id,type,name,image,awards,rankingPosition,priceLevel,priceRange,category,rating,...,latitude,longitude,webUrl,website,rankingString,rankingDenominator,numberOfReviews,review,title,formatted_pricelevel
0,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,25.807375,-80.12364,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,Hands down my absolute favorite hotel in South...,My favorite hotel in Miami--Petar is the best!,relative price : very expensive
1,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,25.807375,-80.12364,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,There are not enough words to explain the sump...,Exquisite Stay,relative price : very expensive
2,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,25.807375,-80.12364,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,I went for the Burlesque show which was absolu...,Burlesqu Show,relative price : very expensive
3,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,25.807375,-80.12364,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,Dana at Gitano made the night magical! He make...,Dana at Gitano is a rockstar!,relative price : very expensive
4,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,25.807375,-80.12364,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,I would like to give our very very heartfelt t...,Perfect Place,relative price : very expensive


In [7]:
import re

def extract_min_max_price(price_range_str):
    # First, let's check if the value is a range
    range_match = re.search(r'\$(\d+[\d,]*)\s*-\s*\$(\d+[\d,]*)', price_range_str)

    if range_match:
        # Extract min and max prices for the range format (ex. $700 - $1000)
        min_price = float(range_match.group(1).replace(',', ''))
        max_price = float(range_match.group(2).replace(',', ''))
        return min_price, max_price
    else:
        # Now let's check for a single value format (ex. $1000)
        single_match = re.search(r'\$(\d+[\d,]*)', price_range_str)
        if single_match:
            single_price = float(single_match.group(1).replace(',', ''))
            return single_price, single_price  # Use the single price as both min and max

        # Return NaN values if no regex patterns match
        return (float('nan'), float('nan'))

df['min_price'], df['max_price'] = zip(*df['priceRange'].apply(extract_min_max_price))
df.head()

,id,type,name,image,awards,rankingPosition,priceLevel,priceRange,category,rating,...,webUrl,website,rankingString,rankingDenominator,numberOfReviews,review,title,formatted_pricelevel,min_price,max_price
0,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,Hands down my absolute favorite hotel in South...,My favorite hotel in Miami--Petar is the best!,relative price : very expensive,729.0,1426.0
1,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,There are not enough words to explain the sump...,Exquisite Stay,relative price : very expensive,729.0,1426.0
2,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,I went for the Burlesque show which was absolu...,Burlesqu Show,relative price : very expensive,729.0,1426.0
3,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,Dana at Gitano made the night magical! He make...,Dana at Gitano is a rockstar!,relative price : very expensive,729.0,1426.0
4,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,I would like to give our very very heartfelt t...,Perfect Place,relative price : very expensive,729.0,1426.0


In [8]:
import spacy

# Load spaCy's English model
nlp = spacy.load("en_core_web_sm")

def preprocess_text(text):
    # Process the review using spaCy
    doc = nlp(text)

    # Extract tokens that are not stopwords, punctuation, or whitespace, then lemmatize them
    tokens = [token.lemma_ for token in doc if not (token.is_stop or token.is_punct or token.is_space)]

    # Join tokens back into a single string and return
    return ' '.join(tokens)

In [9]:
columns_to_clean = ['title', 'review']
df = df.assign(**{f'cleaned_{col}': df[col].apply(preprocess_text) for col in columns_to_clean})

In [10]:
def add_ner_tags(text):
    # Process the text using spaCy NER
    doc = nlp(text)

    # Extract the recognized entities as a list of tags
    tags = [ent.text for ent in doc.ents]

    # Append the tags to the original text
    return ",".join(tags)

df['ner_review'] = df['review'].apply(add_ner_tags)
df['ner_review'][2]

'Burlesque,Patty,one,Tatiana,Patty'

In [11]:
df["text"] = ("title: "+df.cleaned_title.str.strip()+
              "; content: " + df.cleaned_review.str.strip()+
              "; rooms: " + df.numberOfRooms.astype(str)+
              "; ranked: " + df.rankingString.str.strip()+
              "; "+df.formatted_pricelevel+
              "; minimum price per person: "+
              df.min_price.astype(str)+
              "; maximum price per person: "
              +df.max_price.astype(str))
df['text'][2]

'title: Burlesqu; content: go Burlesque absolutely amazing service tier bartender Patty drink great good whiskey sour server Tatiana awesome sweet attentive highly recommend chance ask Patty bar; rooms: 179; ranked: #5 of 235 hotels in Miami Beach; relative price : very expensive; minimum price per person: 729.0; maximum price per person: 1426.0'

In [12]:
df['ner_text'] = df['text'] + "; tags: "+df['ner_review']

In [17]:
sdf = df[['text', 'ner_text', 'name', 'image', 'phone', 'address', 'email', 'priceLevel', 'prices', 'rating', 'latitude', 'longitude', 'website', 'rankingString', 'title', 'review']]

In [ ]:
from sentence_transformers import SentenceTransformer, CrossEncoder, util
import torch

mpnet_embedder = SentenceTransformer('all-mpnet-base-v2')
bge_embedder = SentenceTransformer('BAAI/bge-large-en')

if not torch.cuda.is_available():
    print("Warning: No GPU found. Please add GPU to your notebook")
else:
  print("GPU Found!")
  mpnet_embedder =  mpnet_embedder.to('cuda')
  bge_embedder = bge_embedder.to('cuda')

In [18]:
sdf["mpnet_text_embedding"] = sdf.text.apply(mpnet_embedder.encode)
sdf["mpnet_text_with_ner_embedding"] = sdf.ner_text.apply(mpnet_embedder.encode)

<ipython-input-18-26b633c3c82e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sdf["mpnet_text_embedding"] = sdf.text.apply(mpnet_embedder.encode)
<ipython-input-18-26b633c3c82e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sdf["mpnet_text_with_ner_embedding"] = sdf.ner_text.apply(mpnet_embedder.encode)


In [20]:
sdf["bge_text_embedding"] = sdf.text.apply(lambda x: bge_embedder.encode(x, normalize_embeddings=True))
sdf["bge_text_with_ner_embedding"] = sdf.ner_text.apply(lambda x: bge_embedder.encode(x, normalize_embeddings=True))

<ipython-input-20-40a729a6ac34>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sdf["bge_text_embedding"] = sdf.text.apply(lambda x: bge_embedder.encode(x, normalize_embeddings=True))
<ipython-input-20-40a729a6ac34>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sdf["bge_text_with_ner_embedding"] = sdf.ner_text.apply(lambda x: bge_embedder.encode(x, normalize_embeddings=True))


In [21]:
sdf.to_pickle('pickle/embeddings.pkl')

In [36]:
corpusdf = pd.read_pickle('pickle/embeddings.pkl')

In [37]:
corpusdf.head()

,text,ner_text,name,image,phone,address,email,priceLevel,prices,rating,latitude,longitude,website,rankingString,title,review,mpnet_text_embedding,mpnet_text_with_ner_embedding,bge_text_embedding,bge_text_with_ner_embedding
0,title: favorite hotel Miami Petar good; conten...,title: favorite hotel Miami Petar good; conten...,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,13055348800,"3201 Collins Ave Faena District, Miami Beach, ...",reservations-miamibeach@faena.com,$$$$,[],4.5,25.807375,-80.12364,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,My favorite hotel in Miami--Petar is the best!,Hands down my absolute favorite hotel in South...,"[-0.011474007, 0.010373717, 0.0125973895, 0.01...","[-0.007518154, 0.018034859, 0.0066640046, 0.02...","[0.010578612, 0.012249642, -0.016933508, 0.012...","[0.014103609, 0.009049269, -0.015530758, 0.015..."
1,title: Exquisite stay; content: word explain s...,title: Exquisite stay; content: word explain s...,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,13055348800,"3201 Collins Ave Faena District, Miami Beach, ...",reservations-miamibeach@faena.com,$$$$,[],4.5,25.807375,-80.12364,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,Exquisite Stay,There are not enough words to explain the sump...,"[-0.027685111, -0.005396361, -0.007145659, 0.0...","[-0.031087203, -0.0075291283, -0.02262616, 0.0...","[0.0039651687, 0.018963417, -0.012302323, 0.01...","[0.003249568, 0.015315765, -0.013829733, 0.018..."
2,title: Burlesqu; content: go Burlesque absolut...,title: Burlesqu; content: go Burlesque absolut...,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,13055348800,"3201 Collins Ave Faena District, Miami Beach, ...",reservations-miamibeach@faena.com,$$$$,[],4.5,25.807375,-80.12364,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,Burlesqu Show,I went for the Burlesque show which was absolu...,"[-0.0024554722, 0.0519637, 0.009972114, 0.0100...","[0.008794438, 0.07021385, 0.007254801, -0.0157...","[-0.0012509982, -2.2237562e-05, -0.028536612, ...","[0.0015710691, -0.0040389067, -0.030886017, 0...."
3,title: Dana Gitano rockstar; content: Dana Git...,title: Dana Gitano rockstar; content: Dana Git...,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,13055348800,"3201 Collins Ave Faena District, Miami Beach, ...",reservations-miamibeach@faena.com,$$$$,[],4.5,25.807375,-80.12364,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,Dana at Gitano is a rockstar!,Dana at Gitano made the night magical! He make...,"[0.005952931, 0.045271594, -0.007082046, 0.035...","[0.012021721, 0.05463946, -0.011912774, 0.0241...","[2.4140309e-05, 0.0068328837, -0.029805003, 0....","[0.0037338363, -0.00027515207, -0.031491112, 0..."
4,title: Perfect Place; content: like heartfelt ...,title: Perfect Place; content: like heartfelt ...,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,13055348800,"3201 Collins Ave Faena District, Miami Beach, ...",reservations-miamibeach@faena.com,$$$$,[],4.5,25.807375,-80.12364,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,Perfect Place,I would like to give our very very heartfelt t...,"[-0.041935235, -0.0084601175, -0.0054281056, 0...","[-0.04512196, -0.00079313444, -0.007088178, 0....","[-0.002104935, 0.016902417, -0.003948297, 0.03...","[-0.0007249991, 0.011918714, -0.0044558006, 0...."


In [38]:
query = "closer to the beach, but with a fun family experience. Good for kids but not super expensive."

In [39]:
from openai.embeddings_utils import cosine_similarity
import numpy as np

def search_reviews_mpnet(df, query, n = 15):
  query_embedding = mpnet_embedder.encode(query)
  df["mpnet_text_similarity"] = df.mpnet_text_embedding.apply(lambda x: cosine_similarity(x, query_embedding))
  df["mpnet_ner_text_similarity"] = df.mpnet_text_with_ner_embedding.apply(lambda x: cosine_similarity(x, query_embedding))
  df["mpnet_similarity"] = np.maximum(df['mpnet_text_similarity'], df['mpnet_ner_text_similarity'])
  results = df.sort_values("mpnet_similarity", ascending=False).head(n)
  return results[['name','title', 'review', 'mpnet_text_similarity', 'mpnet_ner_text_similarity', 'mpnet_similarity']]

In [44]:
from openai.embeddings_utils import cosine_similarity
import numpy as np

def search_reviews_bge(df, query, n = 15):
  instruction='Represent this sentence for searching relevant passages:'
  query_embedding = bge_embedder.encode(instruction+query)
  df["bge_text_similarity"] = df.bge_text_embedding.apply(lambda x: cosine_similarity(x, query_embedding))
  df["bge_ner_text_similarity"] = df.bge_text_with_ner_embedding.apply(lambda x: cosine_similarity(x, query_embedding))
  df["bge_similarity"] = np.maximum(df['bge_text_similarity'], df['bge_ner_text_similarity'])
  results = df.sort_values("bge_similarity", ascending=False).head(n)
  return results[['name','title', 'review', 'bge_text_similarity', 'bge_ner_text_similarity', 'bge_similarity']]

In [41]:
search_reviews_mpnet(corpusdf, query)

,name,title,review,mpnet_text_similarity,mpnet_ner_text_similarity,mpnet_similarity
2017,Kimpton Hotel Palomar South Beach,Great alternative to Ocean Drive.,Conclusion:\nI have been in Miami South Beach ...,0.615703,0.617646,0.617646
1394,Sagamore Hotel,Awesome place on the beach at the center of t...,If you are looking for the best value for your...,0.551661,0.583590,0.583590
1137,Marriott Stanton South Beach,February family getaway,Excellent location within walking distance to ...,0.550218,0.576325,0.576325
2181,Loews Miami Beach Hotel,Great Family- Friendly hotel with a resort feel,"Beautiful Pool, View and Family Friendly Hotel...",0.570135,0.573411,0.573411
453,The Palms Hotel & Spa,Perfect beach-and-pool vacation with young kids,"My family and I, including our grandkids aged ...",0.562084,0.569204,0.569204
815,The Meridian Miami,Great location close to the beach and restaura...,"We found the location great, just a few minute...",0.547088,0.554221,0.554221
1402,Sagamore Hotel,"Great location in South Beach, Perfect for Fam...",We just returned from Sagamore Hotel after a w...,0.525354,0.552816,0.552816
257,Hampton Inn Miami Beach - Mid Beach,Miami Beach trip,"If you’re looking for value, this is the place...",0.549765,0.548765,0.549765
1495,Moxy Miami South Beach,Great location at South Beach,A great choice if you come to south beach. Th...,0.516039,0.548845,0.548845
716,Hilton Grand Vacations Club McAlpin Ocean Plaz...,Great Miami Beach Location,"Location, location location. That is what McAl...",0.535005,0.547738,0.547738


In [47]:
bpe_results = search_reviews_bge(corpusdf, query)

In [52]:
bpe_results['review']

2181    Beautiful Pool, View and Family Friendly Hotel...
1674    What an absolutely unbelievable stay, from sta...
880     My family of four (husband, 10 yo boy and 8yo ...
1199    I booked a last minute work trip to Miami and ...
453     My family and I, including our grandkids aged ...
1402    We just returned from Sagamore Hotel after a w...
856     Had a great time in Miami with family, week of...
1107    Great experience and excellent location (with ...
1490    To be clear, the place was great but not for a...
1981    Great euro style hotel on a quiet street in th...
1557    The Carillon Wellness Resort is perfect for a ...
1274    Very nice and clean hotel! It is in a great lo...
887     The staff were amazing.  They anticipated your...
2102    Having stayed beachside  before on Collins Ave...
2199    Stopped to eat after walking the\nBoardwalk. S...
Name: review, dtype: object